In [46]:
import pandas as pd
from pandas import (
    DataFrame, HDFStore
)
import numpy as np

import argparse

In [2]:
df=HDFStore('tmp/pems-bay.h5')

In [2]:
df=pd.read_hdf('tmp/pems-bay.h5')

In [48]:
df=HDFStore('tmp/metr-la.h5')

In [4]:
df=HDFStore('tmp/metr-la2.h5')

In [7]:
df=pd.read_hdf('tmp/metr-la.h5')

ValueError: No dataset in HDF5 file.

In [5]:
print(df)

<class 'pandas.io.pytables.HDFStore'>
File path: tmp/metr-la2.h5



In [49]:
df

<class 'pandas.io.pytables.HDFStore'>
File path: tmp/metr-la.h5

In [6]:
df.close()

In [12]:
df = pd.read_hdf('tmp/pems-bay.h5')
# 0 is the latest observed sample.
x_offsets = np.sort(
    # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
    np.concatenate((np.arange(-11, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 13, 1))
# x: (num_samples, input_length, num_nodes, input_dim)
# y: (num_samples, output_length, num_nodes, output_dim)
x, y = generate_graph_seq2seq_io_data(
    df,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=True,
    add_day_in_week=False,
)

x shape:  (52093, 12, 325, 2) , y shape:  (52093, 12, 325, 2)


In [20]:
x_offsets=x_offsets
y_offsets=y_offsets
add_time_in_day=True
add_day_in_week=False
"""
Generate samples from
:param df:
:param x_offsets:
:param y_offsets:
:param add_time_in_day:
:param add_day_in_week:
:param scaler:
:return:
# x: (epoch_size, input_length, num_nodes, input_dim)
# y: (epoch_size, output_length, num_nodes, output_dim)
"""
num_samples, num_nodes = df.shape
data = np.expand_dims(df.values, axis=-1)
data_list = [data]
if add_time_in_day:
    time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
    time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
    data_list.append(time_in_day)
if add_day_in_week:
    day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
    day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1
    data_list.append(day_in_week)

In [23]:
data.shape

(52116, 325, 1)

In [37]:
np.array(data_list).shape

(2, 52116, 325, 1)

In [39]:
day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1

In [41]:
np.array(day_in_week).shape

(52116, 325, 7)

In [44]:
day_in_week[0:5,0,:]

array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [45]:

data = np.concatenate(data_list, axis=-1)
data.shape

(52116, 325, 2)

In [18]:

data = np.concatenate(data_list, axis=-1)
# epoch_len = num_samples + min(x_offsets) - max(y_offsets)
x, y = [], []
# t is the index of the last observation.
min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = data[t + x_offsets, ...]
    y_t = data[t + y_offsets, ...]
    x.append(x_t)
    y.append(y_t)
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)

In [19]:
print("x shape: ", x.shape, ", y shape: ", y.shape)

x shape:  (52093, 12, 325, 2) , y shape:  (52093, 12, 325, 2)


In [16]:
len(df.columns)

325

In [17]:
max(df.columns)

414694

In [53]:
sensor_ids_filename='tmp/graph_sensor_ids.txt'
distances_filename='tmp/distances_la_2012.csv'
normalized_k=0.1
output_pkl_filename='tmp/adj_mat.pkl'

In [54]:
with open(sensor_ids_filename) as f:
    sensor_ids = f.read().strip().split(',')
distance_df = pd.read_csv(distances_filename, dtype={'from': 'str', 'to': 'str'})
_, sensor_id_to_ind, adj_mx = get_adjacency_matrix(distance_df, sensor_ids)

In [57]:
len(sensor_ids)

207

In [56]:
distance_df

,from,to,cost
0,1201054,1201054,0.0
1,1201054,1201066,2610.9
2,1201054,1201076,2822.7
3,1201054,1201087,2911.5
4,1201054,1201100,7160.1
...,...,...,...
295369,825515,823663,9894.1
295370,825515,825494,475.6
295371,825515,825496,6458.6
295372,825515,825513,6934.2


In [73]:
#def get_adjacency_matrix(distance_df, sensor_ids, normalized_k=0.1):
"""
:param distance_df: data frame with three columns: [from, to, distance].
:param sensor_ids: list of sensor ids.
:param normalized_k: entries that become lower than normalized_k after normalization are set to zero for sparsity.
:return:
"""
num_sensors = len(sensor_ids)
dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
dist_mx[:] = np.inf
# Builds sensor id to index map.
sensor_id_to_ind = {}
for i, sensor_id in enumerate(sensor_ids):
    sensor_id_to_ind[sensor_id] = i

# Fills cells in the matrix with distances.
for row in distance_df.values:
    if row[0] not in sensor_id_to_ind or row[1] not in sensor_id_to_ind:
        continue
    dist_mx[sensor_id_to_ind[row[0]], sensor_id_to_ind[row[1]]] = row[2]

# Calculates the standard deviation as theta.
distances = dist_mx[~np.isinf(dist_mx)].flatten()
std = distances.std()
adj_mx = np.exp(-np.square(dist_mx / std))
# Make the adjacent matrix symmetric by taking the max.
# adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])

# Sets entries that lower than a threshold, i.e., k, to zero for sparsity.
adj_mx[adj_mx < normalized_k] = 0
#return sensor_ids, sensor_id_to_ind, adj_mx

In [ ]:

# Save to pickle file.
with open(args.output_pkl_filename, 'wb') as f:
    pickle.dump([sensor_ids, sensor_id_to_ind, adj_mx], f, protocol=2)

In [59]:
sensor_ids

['773869',
 '767541',
 '767542',
 '717447',
 '717446',
 '717445',
 '773062',
 '767620',
 '737529',
 '717816',
 '765604',
 '767471',
 '716339',
 '773906',
 '765273',
 '716331',
 '771667',
 '716337',
 '769953',
 '769402',
 '769403',
 '769819',
 '769405',
 '716941',
 '717578',
 '716960',
 '717804',
 '767572',
 '767573',
 '773012',
 '773013',
 '764424',
 '769388',
 '716328',
 '717819',
 '769941',
 '760987',
 '718204',
 '718045',
 '769418',
 '768066',
 '772140',
 '773927',
 '760024',
 '774012',
 '774011',
 '767609',
 '769359',
 '760650',
 '716956',
 '769831',
 '761604',
 '717495',
 '716554',
 '773953',
 '767470',
 '716955',
 '764949',
 '773954',
 '767366',
 '769444',
 '773939',
 '774067',
 '769443',
 '767750',
 '767751',
 '767610',
 '773880',
 '764766',
 '717497',
 '717490',
 '717491',
 '717492',
 '717493',
 '765176',
 '717498',
 '717499',
 '765171',
 '718064',
 '718066',
 '765164',
 '769431',
 '769430',
 '717610',
 '767053',
 '767621',
 '772596',
 '772597',
 '767350',
 '767351',
 '716571',

In [60]:
sensor_id_to_ind

{'773869': 0,
 '767541': 1,
 '767542': 2,
 '717447': 3,
 '717446': 4,
 '717445': 5,
 '773062': 6,
 '767620': 7,
 '737529': 8,
 '717816': 9,
 '765604': 10,
 '767471': 11,
 '716339': 12,
 '773906': 13,
 '765273': 14,
 '716331': 15,
 '771667': 16,
 '716337': 17,
 '769953': 18,
 '769402': 19,
 '769403': 20,
 '769819': 21,
 '769405': 22,
 '716941': 23,
 '717578': 24,
 '716960': 25,
 '717804': 26,
 '767572': 27,
 '767573': 28,
 '773012': 29,
 '773013': 30,
 '764424': 31,
 '769388': 32,
 '716328': 33,
 '717819': 34,
 '769941': 35,
 '760987': 36,
 '718204': 37,
 '718045': 38,
 '769418': 39,
 '768066': 40,
 '772140': 41,
 '773927': 42,
 '760024': 43,
 '774012': 44,
 '774011': 45,
 '767609': 46,
 '769359': 47,
 '760650': 48,
 '716956': 49,
 '769831': 50,
 '761604': 51,
 '717495': 52,
 '716554': 53,
 '773953': 54,
 '767470': 55,
 '716955': 56,
 '764949': 57,
 '773954': 58,
 '767366': 59,
 '769444': 60,
 '773939': 61,
 '774067': 62,
 '769443': 63,
 '767750': 64,
 '767751': 65,
 '767610': 66,
 '773

In [61]:
adj_mx

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.3909554, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7174379, 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]], dtype=float32)

In [74]:
dist_mx

array([[    0. ,     inf,     inf, ...,     inf,  8114.8, 10009.7],
       [    inf,     0. ,  2504.6, ...,     inf,     inf,     inf],
       [    inf,  1489.3,     0. , ...,     inf,     inf,  9837. ],
       ...,
       [    inf,     inf,     inf, ...,     0. ,     inf,     inf],
       [ 9599.8,     inf,     inf, ...,     inf,     0. ,     inf],
       [10119.9,  9374.8,     inf, ...,     inf,  9018.7,     0. ]],
      dtype=float32)

In [68]:
mx=np.load('tmp/npy/dist_mx.npy')
#link_mx=np.load('tmp/npy/link_mx.npy')
node_dis=np.load('tmp/npy/node_dis.npy')
node_id=np.load('tmp/npy/node_id.npy')

mx.shape,node_dis.shape,node_id.shape

((106, 106), (106, 3), (106,))

In [69]:
mx

array([[     0.  ,   6100.  ,  12200.  , ..., 178678.1 , 179105.72,
        179582.45],
       [  6100.  ,      0.  ,   6100.  , ..., 172640.92, 173083.48,
        173576.75],
       [ 12200.  ,   6100.  ,      0.  , ..., 166608.34, 167066.88,
        167577.86],
       ...,
       [178678.1 , 172640.92, 166608.34, ...,      0.  ,   3000.  ,
          6000.  ],
       [179105.72, 173083.48, 167066.88, ...,   3000.  ,      0.  ,
          3000.  ],
       [179582.45, 173576.75, 167577.86, ...,   6000.  ,   3000.  ,
             0.  ]], dtype=float32)

In [71]:
node_id

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106'], dtype='<U3')